In [152]:
%load_ext autoreload
%autoreload 2
import traceback
import pyspark
import datetime
import csv
import time
import gzip
import multiprocessing
import pandas as pd
from pprint import pprint
from functools import partial
from pathlib import Path
from typing import Tuple, Optional
import lsde2021.download as dl
import lsde2021.aggregate as agg
from lsde2021.types import PathLike
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType
import pyspark.sql.functions as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
MAX_MEMORY = "60G"

spark = SparkSession \
    .builder \
    .appName("parse-wikipedia-sql-dumps") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .config('spark.ui.showConsoleProgress', 'false') \
    .getOrCreate()
sc = spark.sparkContext

csv_loader = spark.read.format("csv").options(header='True', inferSchema='True')

21/10/09 16:41:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/09 16:41:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [170]:
def parse_wikipedia_sql_dump_handler(
    sql_input_path: PathLike,
    dest: Optional[PathLike] = None,
    force: bool = False
):
    start = time.time()
    csv_config = dict(
        delimiter=',',
        doublequote=False,
        escapechar='\\',
        quotechar="'",
        strict=True,
    )

    input_path = Path(sql_input_path)
    output_path = input_path.with_suffix(".csv")
    if dest is not None:
        output_path = dest / output_path.name
    
    if not force and output_path.exists():
        print(f"using existing {output_path} ...")
        return output_path

    with gzip.open(input_path, mode="rt", encoding="ISO-8859-1") as input_file:
        with open(output_path, mode="w+", encoding="ISO-8859-1") as output_file:
            entries = 0
            try:
                writer = csv.writer(output_file, quoting=csv.QUOTE_MINIMAL)
                for line in input_file.readlines():
                    # Look for INSERT statement and parse it
                    if line.startswith('INSERT INTO'):
                        values = line.partition('` VALUES ')[2]
                        assert values
                        assert values[0] == '('
                        # print("found", values[values.find("79988")-10:][:100])
                        reader = csv.reader([values], **csv_config)
                        for ridx, row in enumerate(reader):
                            for cidx in range(0, len(row), 3):
                                entry = ",".join(row[cidx:cidx+3])[1:-1]
                                try:
                                    entry = tuple(*csv.reader([entry],  delimiter=','))
                                    writer.writerow(entry)
                                    entries += 1
                                except Exception as e:
                                    print("entry:", entry)
                                    print("context:", row[cidx:cidx+10])
                                    raise e
            finally:
                print(f"wrote {entries} rows to {output_path} in {time.time() - start:.2f} seconds ...")
    return output_path

In [171]:
dest = Path("../hdd/wikipedia_sql_dumps")
tables = ["langlinks", "page", "category", "categorylinks"]

languages = pd.read_csv("./data/languages.csv", index_col="code")
languages.head()

,name,dbname,group
code,,,
ar,Arabic,arwiki,ar
ary,Moroccan Arabic,arywiki,ar
arz,Egyptian Arabic,arzwiki,ar
az,Azerbaijani,azwiki,az
azb,South Azerbaijani,azbwiki,az


In [172]:
downloads = list([
    dest / "/".join(dl.wikimedia_sql_dump_local_file(date, wiki=wiki, table=table))
    for (date, wiki, table), _ in dl.wikimedia_sql_dump_urls(
        [datetime.date(2021, 10, 1)], wikis=languages["dbname"], tables=tables
    )
])

print("downloaded %d of %d" % (
    len(list(dest.rglob("**/*sql.gz"))), len(downloads)
))


parsed = sc.parallelize(downloads, numSlices=multiprocessing.cpu_count()).map(
    partial(
        parse_wikipedia_sql_dump_handler,
        force=False,
    )
).collect()

downloaded 204 of 204


using existing ../hdd/wikipedia_sql_dumps/svwiki/20211001/svwiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/dawiki/20211001/dawiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/hiwiki/20211001/hiwiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/viwiki/20211001/viwiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/fiwiki/20211001/fiwiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/hywiki/20211001/hywiki-20211001-langlinks.sql.csv ...

using existing ../hdd/wikipedia_sql_dumps/plwiki/20211001/plwiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/slwiki/20211001/slwiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/kowiki/20211001/kowiki-20211001-langlinks.sql.csv ...using existing ../hdd/wikipedia_sql_dumps/ruwiki/20211001/ruwiki-20211001-langlinks.sql.csv ...

using existing ../hdd/wikipedia_sql_dumps/ltwi

wrote 13024853 rows to ../hdd/wikipedia_sql_dumps/idwiki/20211001/idwiki-20211001-page.sql.csv in 100.73 seconds ...
using existing ../hdd/wikipedia_sql_dumps/idwiki/20211001/idwiki-20211001-category.sql.csv ...
wrote 17011572 rows to ../hdd/wikipedia_sql_dumps/srwiki/20211001/srwiki-20211001-page.sql.csv in 144.72 seconds ...
using existing ../hdd/wikipedia_sql_dumps/srwiki/20211001/srwiki-20211001-category.sql.csv ...
wrote 29213624 rows to ../hdd/wikipedia_sql_dumps/svwiki/20211001/svwiki-20211001-page.sql.csv in 228.74 seconds ...
using existing ../hdd/wikipedia_sql_dumps/svwiki/20211001/svwiki-20211001-category.sql.csv ...
wrote 9100758 rows to ../hdd/wikipedia_sql_dumps/hewiki/20211001/hewiki-20211001-categorylinks.sql.csv in 117.48 seconds ...
using existing ../hdd/wikipedia_sql_dumps/huwiki/20211001/huwiki-20211001-langlinks.sql.csv ...
wrote 5813944 rows to ../hdd/wikipedia_sql_dumps/cswiki/20211001/cswiki-20211001-page.sql.csv in 46.02 seconds ...
using existing ../hdd/wikipe

wrote 26813 rows to ../hdd/wikipedia_sql_dumps/bxrwiki/20211001/bxrwiki-20211001-categorylinks.sql.csv in 0.31 seconds ...
wrote 10010583 rows to ../hdd/wikipedia_sql_dumps/rowiki/20211001/rowiki-20211001-categorylinks.sql.csv in 85.35 seconds ...
wrote 29264559 rows to ../hdd/wikipedia_sql_dumps/zhwiki/20211001/zhwiki-20211001-page.sql.csv in 162.22 seconds ...
using existing ../hdd/wikipedia_sql_dumps/zhwiki/20211001/zhwiki-20211001-category.sql.csv ...
wrote 27093734 rows to ../hdd/wikipedia_sql_dumps/jawiki/20211001/jawiki-20211001-categorylinks.sql.csv in 245.85 seconds ...
wrote 27112846 rows to ../hdd/wikipedia_sql_dumps/ukwiki/20211001/ukwiki-20211001-categorylinks.sql.csv in 255.21 seconds ...
wrote 38996590 rows to ../hdd/wikipedia_sql_dumps/zhwiki/20211001/zhwiki-20211001-categorylinks.sql.csv in 273.82 seconds ...
using existing ../hdd/wikipedia_sql_dumps/ganwiki/20211001/ganwiki-20211001-langlinks.sql.csv ...
wrote 145172 rows to ../hdd/wikipedia_sql_dumps/ganwiki/20211001

In [ ]:
# copy the csv files to the nvme
# join all countries' wiki langlinks with the pageviews so that we know the english titles and can drop the 
# to do this, we have to join with the pages first?

In [169]:
pages = csv_loader.load("../hdd/wikipedia_sql_dumps/dewiki/20211001/dewiki-20211001-page.sql.csv")
pages.show()
print(pages.count(), "rows")

AnalysisException: Path does not exist: file:/home/jovyan/hdd/wikipedia_sql_dumps/dewiki/20211001/dewiki-20211001-page.sql.csv

In [ ]:
# join the pages with their title in english and their english category
# drop all the pages that have no english article

In [109]:
langlinks = csv_loader.load("../nvme/dewiki/dewiki-20211001-langlinks.python.csv")
langlinks.show()
print(langlinks.count(), "rows")

+-------+----+--------------------+
|   from|lang|               title|
+-------+----+--------------------+
|1267329|  aa|                null|
|5755104|  aa|                null|
|6019965|  aa|                null|
|4221663|  aa|                   A|
|9102935|  aa|Category:Candidat...|
| 697833|  aa|    Category:User de|
| 697849|  aa|  Category:User de-1|
| 862809|  aa|      Template:Babel|
| 295773|  aa|Template:Cc-by-sa...|
|4100143|  aa|     Template:Delete|
| 690318|  aa|  Template:User en-3|
| 698291|  aa|  Template:User es-2|
| 697865|  aa|    Template:User fr|
| 697866|  aa|  Template:User fr-1|
| 716292|  aa|    Template:User sk|
|3862540|  aa|     User:Adolfobs93|
|1516974|  aa|       User:Beetstra|
|1813289|  aa|User:CommonsDelinker|
|4550270|  aa|      User:Feudiable|
|3664384|  aa|     User:Groucho NL|
+-------+----+--------------------+
only showing top 20 rows

21878895 rows


In [110]:
pages = csv_loader.load("../nvme/dewiki/dewiki-20211001-page.csv")
pages.show()
print(pages.count(), "rows")

+---+---------+-------------------+-----------+----+
| id|namespace|              title|is_redirect|lang|
+---+---------+-------------------+-----------+----+
|  1|        0|       Alan_Smithee|      false|null|
|  3|        0|           Actinium|      false|null|
|  5|        0|            Ang_Lee|      false|null|
|  7|        0|Anschluss_(Luhmann)|      false|null|
|  8|        0| Anschlussfähigkeit|       true|null|
| 10|        0|      Aussagenlogik|      false|null|
| 11|        0|         Autopoiese|       true|null|
| 12|        0|               A.A.|       true|null|
| 13|        0|Liste_von_Autoren/A|      false|null|
| 14|        0|Liste_von_Autoren/H|      false|null|
| 15|        0|Liste_von_Autoren/C|      false|null|
| 16|        0|Liste_von_Autoren/I|      false|null|
| 17|        0|Liste_von_Autoren/K|      false|null|
| 18|        0|Liste_von_Autoren/J|      false|null|
| 19|        0|Liste_von_Autoren/V|      false|null|
| 20|        0|Liste_von_Autoren/G|      false

In [111]:
all_namespaces = pages.select("namespace").distinct()
all_namespaces.show()
print(all_namespaces.count(), "namespaces")

+---------+
|namespace|
+---------+
|      101|
|       12|
|        1|
|       13|
|        6|
|        3|
|        5|
|       15|
|        9|
|      829|
|        4|
|      100|
|        8|
|        7|
|       10|
|      828|
|       11|
|       14|
|        2|
|        0|
+---------+

20 namespaces


In [112]:
# find the german wikipedia artices with no translation entry
page_langlinks = pages.join(
    langlinks.select([
        F.col("from").alias("id"),
        F.col("lang").alias("translation_lang"),
        F.col("title").alias("translation_title")
    ]), on="id", how="outer")
    # (F.col("id") == F.col("from")), how="outer")
page_langlinks.show()

without_translation = page_langlinks.filter(F.col("translation_lang").isNull())

print("total:", page_langlinks.count())
print("without translation:", without_translation.count())

+---+---------+------+-----------+----+----------------+-----------------+
| id|namespace| title|is_redirect|lang|translation_lang|translation_title|
+---+---------+------+-----------+----+----------------+-----------------+
|148|        0|Ankara|      false|null|              ab|           Анкара|
|148|        0|Ankara|      false|null|             ace|           Ankara|
|148|        0|Ankara|      false|null|             ady|           Анкара|
|148|        0|Ankara|      false|null|              af|           Ankara|
|148|        0|Ankara|      false|null|              ak|           Ankara|
|148|        0|Ankara|      false|null|             als|           Ankara|
|148|        0|Ankara|      false|null|              am|             አንካራ|
|148|        0|Ankara|      false|null|              an|           Ankara|
|148|        0|Ankara|      false|null|             ang|           Ancyra|
|148|        0|Ankara|      false|null|              ar|            أنقرة|
|148|        0|Ankara|   

In [113]:
without_translation.show()

+-----+---------+--------------------+-----------+----+----------------+-----------------+
|   id|namespace|               title|is_redirect|lang|translation_lang|translation_title|
+-----+---------+--------------------+-----------+----+----------------+-----------------+
|  833|        2|           Calle-SUL|      false|null|            null|             null|
| 1591|        1|Geographie_Frankr...|      false|null|            null|             null|
| 2366|        0|                IKRK|       true|null|            null|             null|
| 3918|        0|             Pendeln|      false|null|            null|             null|
| 3997|        0|         Politologie|       true|null|            null|             null|
| 5803|        1|Zehn_Gebote/Archiv/2|      false|null|            null|             null|
| 7993|        4|          Leitlinien|       true|null|            null|             null|
|12046|        1|           Deflation|      false|null|            null|             null|

In [114]:
page_langlinks.filter(F.col("title") == "Pendel").show()
print(page_langlinks.filter(F.col("title") == "Pendel").count())
# Pendel

+-----+---------+------+-----------+----+----------------+------------------+
|   id|namespace| title|is_redirect|lang|translation_lang| translation_title|
+-----+---------+------+-----------+----+----------------+------------------+
|14570|        0|Pendel|      false|null|              ar|              رقاص|
|14570|        0|Pendel|      false|null|             ast|         Pendilexu|
|14570|        0|Pendel|      false|null|              az|            Kəfgir|
|14570|        0|Pendel|      false|null|              be|           Маятнік|
|14570|        0|Pendel|      false|null|              bg|            Махало|
|14570|        0|Pendel|      false|null|             bjn|           Timbatu|
|14570|        0|Pendel|      false|null|              bn|          সরল দোলক|
|14570|        0|Pendel|      false|null|              bs|            Klatno|
|14570|        0|Pendel|      false|null|              ca|            Pèndol|
|14570|        0|Pendel|      false|null|              cs|      

In [115]:
langlinks = csv_loader.load("../nvme/dewiki/wikidatawiki-20211001-langlinks.csv")
langlinks.filter(F.col("from") == "14570").show()
# langlinks.show()

+----+----+-----+
|from|lang|title|
+----+----+-----+
+----+----+-----+



In [58]:
# "../nvme/dewiki/dewiki-20210920-iwlinks.csv"
iwlinks = csv_loader.load("../nvme/dewiki/wikidatawiki-20211001-iwlinks.csv")
iwlinks.filter(F.col("from") == "20702").show()
# iwlinks.show()
print(iwlinks.count(), "rows")

+----+------+-----+
|from|prefix|title|
+----+------+-----+
+----+------+-----+

2047800 rows


In [21]:
# find one translation from a german article
german = df.filter(F.col("lang") == "fr")
print(german.count())
german.show()

0
+----+----+-----+
|from|lang|title|
+----+----+-----+
+----+----+-----+



entries written: 21878895


In [ ]:
#daily = df \
#    .select("domain_code", "page_title", F.col("view_count").alias("view_count2")) \
#    .join(daily, on=["domain_code", "page_title"], how="outer") \
#    .fillna(value=0)
#daily = daily \
#    .withColumn('view_count_sum', sum([daily["view_count"], daily["view_count2"]])) \
#    .select("domain_code", "page_title", F.col("view_count_sum").alias("view_count"))